In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from scipy import stats

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/value_models_combined.csv')
nfl_rolling = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')

In [3]:
rb = nfl[['season', 'week', 'team', 'opponent', 'team_full', 'opponent_full', 'score', 'opponent_score',
          'rushing_value']]
opposing_def = nfl_rolling[['season', 'week', 'team_full', 
          'rush_def_value']]

In [4]:
df = rb.merge(opposing_def, left_on=['season', 'week', 'opponent_full'], 
             right_on=['season', 'week', 'team_full'], suffixes = ('','_opponent'))
df = df.drop(columns=['team_full_opponent'])

In [5]:
def value_adjustment(value, adjusting_value, original_value_percentage=.95, adjustment_threshold=.1):
    value_over_expected = value + adjusting_value
    adjusted_value = (value_over_expected*adjustment_threshold) + (value*original_value_percentage)
    adjustment = adjusted_value - value
    
    return adjustment, adjusted_value

df[['rushing_adjustment', 'rushing_adjusted_value']] = df.apply(lambda x: value_adjustment(x.rushing_value, x.rush_def_value, 
                                original_value_percentage=.7, adjustment_threshold=.3), axis=1, result_type='expand')

In [6]:
df.to_csv('rush_offense_value_modeling_adjusted_data/rushing_offense_values_with_adjustment.csv')

In [7]:
testing = df.dropna()
testing.sort_values(by='rushing_adjustment', ascending=False)

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,rushing_value,rush_def_value,rushing_adjustment,rushing_adjusted_value
46,2014,2,JAX,WAS,Jacksonville Jaguars,Washington Football Team,10,41,-0.396661,1.529695,0.458908,0.062247
55,2014,2,NYG,ARI,New York Giants,Arizona Cardinals,14,25,-1.256726,1.468394,0.440518,-0.816208
115,2014,4,NYJ,DET,New York Jets,Detroit Lions,17,24,-0.025183,1.407627,0.422288,0.397105
117,2014,4,PIT,TB,Pittsburgh Steelers,Tampa Bay Buccaneers,24,27,-0.492622,1.367205,0.410161,-0.082460
92,2014,3,SF,ARI,San Francisco 49ers,Arizona Cardinals,14,23,-0.122913,1.362560,0.408768,0.285855
...,...,...,...,...,...,...,...,...,...,...,...,...
3809,2021,3,LAC,KC,Los Angeles Chargers,Kansas City Chiefs,30,24,-0.172352,-1.407762,-0.422329,-0.594680
2304,2018,6,NE,KC,New England Patriots,Kansas City Chiefs,43,40,1.705721,-1.449878,-0.434963,1.270758
34,2014,2,BAL,PIT,Baltimore Ravens,Pittsburgh Steelers,26,6,0.376348,-1.583139,-0.474942,-0.098594
56,2014,2,NYJ,GB,New York Jets,Green Bay Packers,24,31,0.565160,-1.609059,-0.482718,0.082443
